In [ ]:
# review -> 이모티콘/자음 삭제 -> (띄워쓰기 + 맞춤법) -> 문장 분리
# - 문장 단위: 문장 부호, 종결어미, 연결조사로 사전 만들기

In [65]:
import pandas as pd
from kiwipiepy import Kiwi
from emoji import core
import re

In [66]:
data = pd.read_csv('./data/train_data_1.csv', encoding='utf-8-sig')
data.head()

,review
0,맛있어요~
1,만족도 최상의 커피맛집
2,찐 커피맛집ㅎㅎ
3,로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!! 마...
4,커조남 커피는 힐링입니다 소금빵도 맛있어요


In [67]:
kiwi = Kiwi()

In [68]:
# 사용자 사전 추가
kiwi.add_user_word('맛집', 'NNG', 0)
kiwi.add_user_word('카페명', 'NNP', 0) # 카페 이름 리스트가 있으면 추가

True

In [69]:
# 정규표현식 컴파일 (명사형 태그)
noun = re.compile(r'^N')

In [70]:
# 리뷰 문장 단위로 분리
def revToSent(review):
    sentences = []
    preTag = ''
    preForm = ''
    sents = kiwi.split_into_sents(review, return_tokens=True)
    for sent in sents:
        toks = sent.tokens
        tmp = []
        for tok in toks:
            if tok.tag == 'SW':
                continue
            
            condition = (preTag == 'EC') and (preForm in ['고', '지만'])
            if condition and bool(noun.match(tok.tag)):
                sentences.append(kiwi.join(tmp))
                tmp.clear()

            tmp.append(tok)
            
            
            if tok.tag == 'SO' and tok.form == '~':
                sentences.append(kiwi.join(tmp))
                tmp.clear()
            
            preTag = tok.tag
            preForm = tok.form

        if len(tmp) > 0:
            sentences.append(kiwi.join(tmp))
    
    return sentences

In [72]:
# 물결표 ... 고민해보기
for i in range(15, 25):
    review = data['review'][i]
    print('원본 문장: ', review)
    review = core.replace_emoji(review, replace="")
    print('특수 문자 제거: ', review)
    sents = revToSent(review)
    print('문장 분리: ', sents)
    print()    

원본 문장:  드립커피는 별로 안좋아해서..
친구는 괜찮대요. 갈 만은 해요
특수 문자 제거:  드립커피는 별로 안좋아해서..
친구는 괜찮대요. 갈 만은 해요
문장 분리:  ['드립커피는 별로 안 좋아해서..', '친구는 괜찮대요.', '갈 만은해요']

원본 문장:  두번째 방문입니다.일부러 들렀어요. 전에 마셔보지못한 카페오레가 궁금해서요. 카페오레 온도가 참좋아요. 부드러움도요. 소금빵과 잘어울리는 맛입니다.
특수 문자 제거:  두번째 방문입니다.일부러 들렀어요. 전에 마셔보지못한 카페오레가 궁금해서요. 카페오레 온도가 참좋아요. 부드러움도요. 소금빵과 잘어울리는 맛입니다.
문장 분리:  ['두 번째 방문입니다.', '일부러 들렀어요.', '전에 마셔 보지 못한 카페오레가 궁금해서요.', '카페오레 온도가 참 좋아요.', '부드러움도요.', '소금 빵과 잘 어울리는 맛입니다.']

원본 문장:  커조남☕️모르시는 대구분들 계신가용~? 동성로점도 넘 좋았지만 본점은 정말 최고인 것 같아요 평일에도 항상 사람들 많고 직원분들 모두 친절하시고 무엇보다 커피맛 정말 최고에요! 디카페인 추가금액 없는곳 커조남 뿐일것같아요! 그리고 리필된다는점~진짜 넘 좋은곳!추천합니다!⭐️⭐️⭐️⭐️⭐️
특수 문자 제거:  커조남모르시는 대구분들 계신가용~? 동성로점도 넘 좋았지만 본점은 정말 최고인 것 같아요 평일에도 항상 사람들 많고 직원분들 모두 친절하시고 무엇보다 커피맛 정말 최고에요! 디카페인 추가금액 없는곳 커조남 뿐일것같아요! 그리고 리필된다는점~진짜 넘 좋은곳!추천합니다!
문장 분리:  ['커조남모르 시는 대구부인들 계신가용~', '?', '동성로 점도 넘 좋았지만', '본점은 정말 최고인 것 같아요', '평일에도 항상 사람들 많고', '직원 분들 모두 친절하시고', '무엇보다 커피 맛 정말 최고에요!', '디 카페인 추가 금액 없는 곳 커조 남 뿐일 것 같아요!', '그리고 리필된다는 점~', '진짜 넘 좋은 곳!', '추천합니다!']

원본 문장:  카페오레 맛